In [ ]:
import random as ran
import jacinle.random as random
import numpy as np
import json
import openai
from sklearn.metrics import f1_score
import os
from openai import AzureOpenAI, OpenAI
import family_tree

n=10 #adjustable to 20 and 40
Data_prefix = f'Data_{n}/'
run = 100
file_name = 'Fam'

In [ ]:
def Family_tree_GPT4o(name, n, run):
    if name == 'sisters':
        prefix = f'GPT_4o_{n}/Sisters/' 
    elif name = 'grandsons':
        prefix = f'GPT_4o_{n}/Grandsons/' 
    elif name == 'aunts':
        prefix = f'GPT_4o_{n}/Aunts/'
    elif name = 'paternal_great_aunts':
        prefix = f'GPT_4o_{n}/Paternal_great_aunts/' 
    else:
        print('Error in name!')
        return 0

    F1 = []
    Data_prefix = f'Data_{n}/'

    for itr in range(0,run):

        fathers_arr = np.load(Data_prefix+file_name+'_fathers_arr_'+str(itr)+'.npy')
        mothers_arr = np.load(Data_prefix+file_name+'_mothers_arr_'+str(itr)+'.npy')
        daughters_arr = np.load(Data_prefix+file_name+'_daughters_arr_'+str(itr)+'.npy')
        sons_arr = np.load(Data_prefix+file_name+'_sons_arr_'+str(itr)+'.npy')

        relations = family_tree.generate_relations(n, fathers_arr, mothers_arr, daughters_arr, sons_arr)
        sys_prompt = family_tree.create_system_prompt()
        
        if name == 'sisters':
            sisters_arr = np.load(Data_prefix+file_name+'_sisters_arr_'+str(itr)+'.npy')
            user_prompt_sisters = family_tree.create_user_prompt_sisters(n, relations)
        elif name = 'grandsons':
            grandsons_arr = np.load('Data/'+file_name+'_grandsons_arr_'+str(itr)+'.npy')
            user_prompt_grandsons = create_user_prompt_grandsons(n, relations)
        elif name == 'aunts':
            aunts_arr = np.load('Data/'+file_name+'_aunts_arr_'+str(itr)+'.npy')
            user_prompt_aunts = create_user_prompt_aunts(n, relations)
        elif name = 'paternal_great_aunts':
            paternal_great_aunts_arr = np.load(Data_prefix+file_name+'_paternal_great_aunts_arr_'+str(itr)+'.npy')
            user_prompt_paternal_great_aunts = create_user_prompt_paternal_great_aunts(n, relations)

        client = family_tree.create_client_gpt('YOUR API KEY',"2025-01-01-preview",'https://trial-4o.openai.azure.com/')
        llm_response = family_tree.prompt_llm_gpt_4o(client, sys_prompt, eval('user_prompt_'+name))

        try:
            json_output, output_matrix = family_tree.extract_json_and_numpy_array(llm_response)
        except:
            try:
                response = llm_response.choices[0].message.content
                response = response.replace("```json", "")
                response = response.replace("```", "")
                json_output =  {'content': response}
                json_output = json.dumps(json_output)
            except:
                pass

        try:
            f1 = family_tree.cal_f1_score(eval(name+'_arr'), output_matrix, weight_method='micro')
            json_output.update({'F1': f1})
            F1.append(f1)
        except:                   
            pass 

        try:
            with open(prefix + file_name + '_' + str(itr) + '_' + name + '.json', 'w', encoding='utf-8') as f:
                json.dump(json_output, f, ensure_ascii=False, indent=4)
        except:                   
            pass
    

In [ ]:
def Family_tree_DSv3(name, n, run):
    if name == 'sisters':
        prefix = f'DS_v3_{n}/Sisters/' 
    elif name = 'grandsons':
        prefix = f'DS_v3_{n}/Grandsons/' 
    elif name == 'aunts':
        prefix = f'DS_v3_{n}/Aunts/'
    elif name = 'paternal_great_aunts':
        prefix = f'DS_v3_{n}/Paternal_great_aunts/' 
    else:
        print('Error in name!')
        return 0

    F1 = []
    Data_prefix = f'Data_{n}/'

    for itr in range(0,run):

        fathers_arr = np.load(Data_prefix+file_name+'_fathers_arr_'+str(itr)+'.npy')
        mothers_arr = np.load(Data_prefix+file_name+'_mothers_arr_'+str(itr)+'.npy')
        daughters_arr = np.load(Data_prefix+file_name+'_daughters_arr_'+str(itr)+'.npy')
        sons_arr = np.load(Data_prefix+file_name+'_sons_arr_'+str(itr)+'.npy')

        relations = family_tree.generate_relations(n, fathers_arr, mothers_arr, daughters_arr, sons_arr)
        sys_prompt = family_tree.create_system_prompt()
        
        if name == 'sisters':
            sisters_arr = np.load(Data_prefix+file_name+'_sisters_arr_'+str(itr)+'.npy')
            user_prompt_sisters = family_tree.create_user_prompt_sisters(n, relations)
        elif name = 'grandsons':
            grandsons_arr = np.load('Data/'+file_name+'_grandsons_arr_'+str(itr)+'.npy')
            user_prompt_grandsons = create_user_prompt_grandsons(n, relations)
        elif name == 'aunts':
            aunts_arr = np.load('Data/'+file_name+'_aunts_arr_'+str(itr)+'.npy')
            user_prompt_aunts = create_user_prompt_aunts(n, relations)
        elif name = 'paternal_great_aunts':
            paternal_great_aunts_arr = np.load(Data_prefix+file_name+'_paternal_great_aunts_arr_'+str(itr)+'.npy')
            user_prompt_paternal_great_aunts = create_user_prompt_paternal_great_aunts(n, relations)

        client = family_tree.create_client_ds('YOUR API KEY',"https://api.deepseek.com")
        llm_response = family_tree.prompt_llm_ds(client, sys_prompt, eval('user_prompt_'+name))

        try:
            json_output, output_matrix = family_tree.extract_json_and_numpy_array(llm_response)
        except:
            try:
                response = llm_response.choices[0].message.content
                response = response.replace("```json", "")
                response = response.replace("```", "")
                json_output =  {'content': response}
                json_output = json.dumps(json_output)
            except:
                pass

        try:
            f1 = family_tree.cal_f1_score(eval(name+'_arr'), output_matrix, weight_method='micro')
            json_output.update({'F1': f1})
            F1.append(f1)
        except:                   
            pass 

        try:
            with open(prefix + file_name + '_' + str(itr) + '_' + name + '.json', 'w', encoding='utf-8') as f:
                json.dump(json_output, f, ensure_ascii=False, indent=4)
        except:                   
            pass
    

In [ ]:
def Family_tree_DSr1(name, n, run):
    if name == 'sisters':
        prefix = f'DS_r1_{n}/Sisters/' 
    elif name = 'grandsons':
        prefix = f'DS_r1_{n}/Grandsons/' 
    elif name == 'aunts':
        prefix = f'DS_r1_{n}/Aunts/'
    elif name = 'paternal_great_aunts':
        prefix = f'DS_r1_{n}/Paternal_great_aunts/' 
    else:
        print('Error in name!')
        return 0

    F1 = []
    Data_prefix = f'Data_{n}/'

    for itr in range(0,run):

        fathers_arr = np.load(Data_prefix+file_name+'_fathers_arr_'+str(itr)+'.npy')
        mothers_arr = np.load(Data_prefix+file_name+'_mothers_arr_'+str(itr)+'.npy')
        daughters_arr = np.load(Data_prefix+file_name+'_daughters_arr_'+str(itr)+'.npy')
        sons_arr = np.load(Data_prefix+file_name+'_sons_arr_'+str(itr)+'.npy')

        relations = family_tree.generate_relations(n, fathers_arr, mothers_arr, daughters_arr, sons_arr)
        sys_prompt = family_tree.create_system_prompt()
        
        if name == 'sisters':
            sisters_arr = np.load(Data_prefix+file_name+'_sisters_arr_'+str(itr)+'.npy')
            user_prompt_sisters = family_tree.create_user_prompt_sisters(n, relations)
        elif name = 'grandsons':
            grandsons_arr = np.load('Data/'+file_name+'_grandsons_arr_'+str(itr)+'.npy')
            user_prompt_grandsons = create_user_prompt_grandsons(n, relations)
        elif name == 'aunts':
            aunts_arr = np.load('Data/'+file_name+'_aunts_arr_'+str(itr)+'.npy')
            user_prompt_aunts = create_user_prompt_aunts(n, relations)
        elif name = 'paternal_great_aunts':
            paternal_great_aunts_arr = np.load(Data_prefix+file_name+'_paternal_great_aunts_arr_'+str(itr)+'.npy')
            user_prompt_paternal_great_aunts = create_user_prompt_paternal_great_aunts(n, relations)

        client = family_tree.create_client_ds('YOUR API KEY',"https://api.deepseek.com")
        llm_response = family_tree.prompt_llm_ds_r1(client, sys_prompt, eval('user_prompt_'+name))

        try:
            json_output, output_matrix = family_tree.extract_json_and_numpy_array(llm_response)
        except:
            try:
                response = llm_response.choices[0].message.content
                response = response.replace("```json", "")
                response = response.replace("```", "")
                json_output =  {'content': response}
                json_output = json.dumps(json_output)
            except:
                pass

        try:
            f1 = family_tree.cal_f1_score(eval(name+'_arr'), output_matrix, weight_method='micro')
            json_output.update({'F1': f1})
            F1.append(f1)
        except:                   
            pass 

        try:
            with open(prefix + file_name + '_' + str(itr) + '_' + name + '.json', 'w', encoding='utf-8') as f:
                json.dump(json_output, f, ensure_ascii=False, indent=4)
        except:                   
            pass
    

In [ ]:
# Example: Evaluating the family tree problem of 'sisters' using Deepseek-r1 with n=10 in 100 iterations
Family_tree_DSr1('sisters', 10, 100)